# **Procura de Problemas Éticos em Issues**

## **Imports necessários para o funcionamento:**

In [ ]:
%pip install pandas
%pip install nltk
%pip install ipywidgets


import pandas as pd
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import ipywidgets as widgets
from IPython.display import display
from collections import Counter

## **Fazendo a leitura dos dados:**

In [ ]:
issues = pd.read_csv("path_to_csv")
comentarios = pd.read_csv("path_to_csv")

# Concatenando o Titulo e a Descrição das issues
issues['Info'] = issues['TituloIssue'] + issues['DescricaoIssue']

# Convertendo todos os tipos para Strings minusculas
issues['Info'] = issues['Info'].astype(str).str.lower()
comentarios['Comentario'] = comentarios['Comentario'].astype(str).str.lower()

## **Tokenizando os Comentarios, Titulo e Descrição das Issues**

In [ ]:
# Definindo as StopWords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

# Função que remove as stopwords
def removerStopWords(palavras):
    return [palavra for palavra in palavras if palavra not in stopWords and palavra.isalpha() or palavra in expressoes]

# Criando um tokenizer que considera expressões
expressoes = {'data governance', 'data protection','informed consent', 'lack of data', 'user data collection' 'business ethics','conflict of interest',
            'human agency', 'intellectual property','regulatory approaches', 'self-conception','professional ethics', 'work ethics','common goods','individual differences',
            'non-discrimination', 'non-maleficence','prevention of harm', 'quality of life', 'respect for human autonomy', 'retention and addiction', 'social justice',
            'speech issues', 'technical robustness','computer abuse', 'malicious use'}

expressaoRegular = r'\b(?:' + '|'.join(map(re.escape,expressoes)) + r')\b|\w+'
tokenizer = RegexpTokenizer(expressaoRegular)


# Tokenize dos comentarios das issues
comentariosToK = []
for i in range (comentarios['Comentario'].size):
    comentariosToK.append(removerStopWords(tokenizer.tokenize(comentarios.loc[i]['Comentario'])))

# Tokenize do campo Info
tituloDescricaoToK = []
for k in range(issues['Info'].size):
    tituloDescricaoToK.append(removerStopWords(tokenizer.tokenize(issues.loc[k]['Info'])))

## **Preparando o Dataframe com os termos relacionados à ética:**

In [ ]:
#Definindo os EthicalIssues
EthicalIssuesOtavio = ['bias', 'data', 'encryption', 'monetization', 'openness', 'privacy', 'authorship', 'autonomy', 'beneficence', 'commerce', 'compliance', 'confidentiality',
                 'context', 'dependability', 'fairness', 'oversight', 'responsibility', 'trust', 'trustworthiness', 'axiology', 'freedom', 'solidarity', 'utility',
                 'care', 'competence', 'access', 'accessibility', 'dignity', 'diversity', 'equality', 'equity', 'humanity', 'inclusiveness', 'inequality', 'justice', 
                 'participation', 'plurality', 'sustainability', 'unemployment', 'welfare', 'accountability', 'accuracy', 'anonymity', 'comprehensibility', 'consistency',
                 'contestability', 'explainability', 'explicability', 'integrity', 'interpretability', 'liability', 'reliability', 'safety', 'security', 'traceability', 'transparency',
                 'usability', 'data governance', 'data protection', 'informed consent', 'lack of data', 'user data collection', 'business ethics', 'conflict of interest', 
                 'human agency', 'intellectual property', 'regulatory approaches', 'professional ethics', 'individual differences', 'self-conception', 'work ethics', 
                 'common goods', 'non-discrimination', 'non-maleficence', 'prevention of harm', 'quality of life', 'respect for human autonomy', 'retention and addiction', 
                 'social justice', 'speech issues', 'technical robustness', 'computer abuse', 'malicious use']

EthicalIssuesBase = ['fairness', 'bias', 'safety', 'harmful content', 'toxicity', 'hallucinations', 'privacy', 'data protection', 'interaction risks', 'security', 'robustness', 
                 'education', 'learning', 'aligment', 'cybercrime', 'governance', 'regulation', 'labor displacement', 'economic impact', 'transparency', 'explainability', 
                 'evaluation', 'auditing', 'sustainability', 'art', 'creativity', 'copyright', 'autorship', 'writing', 'research', 'miscellaneous']

EthicalIssuesWordnet = ['fairness', 'bias', 'fairly', 'equity', 'preconception', 'safety', 'safe', 'harmful content', 'toxicity', 'toxic', 'harm', 'risk', 'authoriativeness',
                        'sensitivity', 'misinformation', 'inaccuracies', 'privacy', 'data protection', 'privateness', 'secrecy', 'interaction risks', 'overreliance',
                        'manipulation', 'unethical actions', 'dependency', 'lack of empaty', 'security', 'robustness', 'protection', 'validity', 'hardiness', 'cybercrime', 'attack',
                        'fraud', 'governance', 'regulation', 'administration', 'regularisation', 'regulating', 'economic impact', 'effect', 'economical', 'transparency',
                        'explainability', 'transparence', 'transparentness', 'evaluation', 'auditing', 'rating', 'valuating', 'audit', 'inspect', 'sustainability', 'renewable energy',
                        'copyright', 'authorship', 'plagiarism', 'accountability', 'responsibly', 'trustworthiness', 'moral views']

EthicalIssues = list(set(EthicalIssuesWordnet + EthicalIssuesOtavio))


# Criando o DataFrame que guardará as ocorrências dos comentários
ocorrenciasComents = pd.DataFrame()
ocorrenciasComents['EthicalIssues'] = EthicalIssues
ocorrenciasComents['Ocorrencias'] = 0
ocorrenciasComents['PalavrasAntes'] = ""
ocorrenciasComents['PalavrasDepois'] = ""
ocorrenciasComents = ocorrenciasComents.set_index('EthicalIssues')

# Criando o DataFrame que guardará as ocorrências do título e descrição
ocorrenciasTituloDescricao = pd.DataFrame()
ocorrenciasTituloDescricao['EthicalIssues'] = EthicalIssues
ocorrenciasTituloDescricao['Ocorrencias'] = 0
ocorrenciasTituloDescricao['PalavrasAntes'] = ""
ocorrenciasTituloDescricao['PalavrasDepois'] = ""
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.set_index('EthicalIssues')

## **Contando a ocorrência dos termos:**

In [ ]:
# Função que faz a contagem das ocorrências dos termos e conta as palavras antes e depois
def contaOcorrencias(tokens, dfOcorrencias):
    for listaTokens in tokens:
        for index, token in enumerate(listaTokens):
            if token in dfOcorrencias.index:
                
                # Adicionando uma ocorrência do token
                dfOcorrencias.loc[token, 'Ocorrencias'] += 1
                
                if (index > 0):
                    # Adicionando a palavra anterior se existir
                    if (dfOcorrencias.loc[token, 'PalavrasAntes'] == ""):
                        dfOcorrencias.loc[token, 'PalavrasAntes'] = listaTokens[index - 1]
                    else:
                       dfOcorrencias.loc[token, 'PalavrasAntes']  += ("\n" + listaTokens[index - 1])
                    
                if (index < len(listaTokens) - 1):
                    # Adicionando a palavra depois se existir
                    if (dfOcorrencias.loc[token, 'PalavrasDepois'] == ""):
                        dfOcorrencias.loc[token, 'PalavrasDepois'] = listaTokens[index + 1]
                    else:
                       dfOcorrencias.loc[token, 'PalavrasDepois']  += ("\n" + listaTokens[index + 1])

contaOcorrencias(comentariosToK, ocorrenciasComents)
contaOcorrencias(tituloDescricaoToK, ocorrenciasTituloDescricao)

## **Deixando o dataframe no formato correto:**

In [ ]:
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.reset_index()
ocorrenciasComents = ocorrenciasComents.reset_index()

ocorrenciasTituloDescricao['PalavrasAntes'] = ocorrenciasTituloDescricao['PalavrasAntes'].str.split('\n')
ocorrenciasTituloDescricao['PalavrasDepois'] = ocorrenciasTituloDescricao['PalavrasDepois'].str.split('\n')

ocorrenciasComents['PalavrasAntes'] = ocorrenciasComents['PalavrasAntes'].str.split('\n')
ocorrenciasComents['PalavrasDepois'] = ocorrenciasComents['PalavrasDepois'].str.split('\n')

ocorrenciasComents = ocorrenciasComents[ocorrenciasComents['Ocorrencias'] > 0]
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao[ocorrenciasTituloDescricao['Ocorrencias'] > 0]

ocorrenciasComents = ocorrenciasComents.sort_values(by='Ocorrencias', ascending=False)
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.sort_values(by='Ocorrencias', ascending=False)

## **Criando uma tabela expansivel**

In [ ]:
# Função para criar a tabela com linhas expansíveis e contagem de palavras
def criarTabelaExpansivel(df):
    displayWidgets = []
    
    for _, linha in df.iterrows():
        # Conta as palavras em 'palavrasAntes' e 'palavrasDepois'
        antes_counts = Counter(linha['PalavrasAntes']).most_common()  # Ordena por contagem (decrescente)
        depois_counts = Counter(linha['PalavrasDepois']).most_common()  # Ordena por contagem (decrescente)
        
        # Cria uma linha com os dados principais
        linha_widget = widgets.HBox([
            widgets.Label(value=str(linha["EthicalIssues"]), layout=widgets.Layout(width="120px")),
            widgets.Label(value=str(linha["Ocorrencias"]), layout=widgets.Layout(width="60px")),
            widgets.Label(value="Clique para expandir", layout=widgets.Layout(width="200px"))
        ])
        
        # Cria o conteúdo expansível com `palavrasAntes` e `palavrasDepois` e suas contagens
        widgetDetalhes = widgets.VBox([
            widgets.Label(value=f"Palavras Antes: {', '.join([f'{word} ({count})' for word, count in antes_counts])}"),
            widgets.Label(value=f"Palavras Depois: {', '.join([f'{word} ({count})' for word, count in depois_counts])}")
        ])
        
        # Torna o conteúdo expansível
        expansao = widgets.Accordion(children=[widgetDetalhes])
        expansao.set_title(0, f"Detalhes de '{linha['EthicalIssues']}'")
        
        # Agrupa a linha principal e o conteúdo expansível
        linha_box = widgets.VBox([linha_widget, expansao])
        displayWidgets.append(linha_box)
    
    # Exibe a tabela com todas as linhas e detalhes
    display(widgets.VBox(displayWidgets))

## **Exibindo os resultados:**

In [ ]:
# Exibe a tabela expansível
criarTabelaExpansivel(ocorrenciasComents)


In [ ]:
# Exibe a tabela expansível
criarTabelaExpansivel(ocorrenciasTituloDescricao)

# **Exportando as Tabelas**

In [ ]:
# Exportando a tabela expansível como um arquivo html

def exportar_para_html(df, filepath):
    html = """<html>
    <head>
        <meta charset="UTF-8">
        <style>
            table {width: 100%%; border-collapse: collapse;}
            th, td {border: 1px solid black; padding: 8px; text-align: left;}
            th {background-color: #f2f2f2;}
            .detalhes {display: none;}
        </style>
        <script>
            function toggleDetalhes(id) {
                var elemento = document.getElementById(id);
                elemento.style.display = (elemento.style.display === "none") ? "block" : "none";
            }
        </script>
    </head>
    <body>
        <h2>Tabela Expansível</h2>
        <table>
            <tr>
                <th>Ethical Issues</th>
                <th>Ocorrências</th>
                <th>Detalhes</th>
            </tr>
    """

    for i, (_, linha) in enumerate(df.iterrows()):
        antes_counts = Counter(linha["PalavrasAntes"]).most_common()
        depois_counts = Counter(linha["PalavrasDepois"]).most_common()

        detalhes = f"""<p><strong>Palavras Antes:</strong> {', '.join([f'{word} ({count})' for word, count in antes_counts])}</p>
                       <p><strong>Palavras Depois:</strong> {', '.join([f'{word} ({count})' for word, count in depois_counts])}</p>"""

        html += f"""
            <tr>
                <td>{linha["EthicalIssues"]}</td>
                <td>{linha["Ocorrencias"]}</td>
                <td><button onclick="toggleDetalhes('detalhes{i}')">Expandir</button></td>
            </tr>
            <tr id="detalhes{i}" class="detalhes">
                <td colspan="3">{detalhes}</td>
            </tr>
        """

    html += "</table></body></html>"

    with open(filepath, "w", encoding="utf-8") as f:
        f.write(html)


In [ ]:
exportar_para_html(ocorrenciasComents, "path_to_html")
exportar_para_html(ocorrenciasTituloDescricao, "path_to_html")